Zde hledáme Wikidata IDs toho autorstva, které má ve vyfiltrovaných datech ČNB (knihy > r. 1900) alespoň 2 záznamy.

In [69]:
import os
import re
import json
import requests
import pandas as pd

In [70]:
kam = 'data_raw/wikidata_doplneni'
if not os.path.exists(kam):
    os.makedirs(kam)

In [71]:
vsechny_jsony = [f for f in os.listdir(kam)]

In [72]:
vyhledane = set([x.split("_")[0] for x in vsechny_jsony])

In [73]:
df = pd.read_parquet(os.path.join("data","cnb_vyber.parquet"))

In [74]:
df = df.explode('100_7')

In [75]:
df['100_7'].nunique()

149542

In [76]:
df.groupby('100_7').size().nlargest(1000)

100_7
jn20001103529    1652
jn20000810141    1180
jk01051816       1013
jk01083016        946
jk01021023        812
                 ... 
jk01091051         50
jk01100905         50
jk01120617         50
jk01121525         50
jk01121770         50
Length: 1000, dtype: int64

In [77]:
opakovane = df.groupby('100_7').size()
opakovane = opakovane.sort_values(ascending=False)
opakovane = opakovane[opakovane > 1].index.to_list()
len(opakovane)

76782

In [78]:
autority = pd.read_parquet(os.path.join("data","aut_vyber.parquet"))

In [79]:
autority

,024_2,024_a,046_f,046_g,100_7,100_a,100_d,100_ind1,110_a,370_a,...,411_a,430_a,450_a,500_a,500_i,500_ind1,550_7,678_a,856_u,leader
001,,,,,,,,,,,,,,,,,,,,,
jk01010001,None,None,[1931],None,[jk01010001],"[Abraham, Jiří,]",[1931-],[1],None,None,...,None,None,None,None,None,None,None,"[Narozen 14.12.1931 v Osvračíně. MUDr., odborn...",None,nz a22 n 4500
jk01010002,None,None,[1890],[1933],[jk01010002],"[Abraham, Josef,]",[1890-1933],[1],None,None,...,None,None,None,None,None,None,None,[Narozen 19.11.1890 v Osvračíně u Horšova Týna...,None,nz a22 n 4500
jk01010003,None,None,[1957],None,[jk01010003],"[Abraham, Pavel,]",[1957-],[1],None,None,...,None,None,None,None,None,None,None,[Narozen roku 1957. Student filosofické fakult...,None,nz a22 n 4500
jk01010004,None,None,[1889],[1963],[jk01010004],"[Absolon, Josef,]",[1889-1963],[1],None,"[Odrlice, Senice na Hané, Česko]",...,None,None,None,None,None,None,None,"[Narozen 27. 1. 1889 v Odrlicích na Moravě, ze...",None,cz a22 n 4500
jk01010005,"[isni, wikidata]","[0000000109542912, Q1063371]",[1877],[1960],[jk01010005],"[Absolon, Karel,]",[1877-1960],[1],None,None,...,None,None,None,None,None,None,None,"[Narozen 16.6.1877 v Boskovicích, zemřel 6.10....",None,cz a22 n 4500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
xx0323367,None,None,None,None,[xx0323367],"[Birhanu, Abigail]",None,[1],None,None,...,None,None,None,None,None,None,None,[Americká výtvarnice a učitelka výtvarné výcho...,None,nz a22 n 4500
xx0323368,None,None,None,None,[xx0323368],"[Latimer, Khadesia]",None,[1],None,None,...,None,None,None,None,None,None,None,[Americká výtvarnice a výtvarná pedagožka zamě...,None,nz a22 n 4500
xx0323369,None,None,None,None,[xx0323369],"[Liz, Paula]",None,[1],None,None,...,None,None,None,None,None,None,None,[Americká výtvarnice a výtvarná pedagožka.],None,nz a22 n 4500


In [80]:
autority = autority[autority['024_2'].notnull()]
autority = autority[autority['024_a'].notnull()]

In [81]:
autority = autority.explode(['024_a','024_2'])

In [82]:
autority_s_wikidaty = set(autority[autority['024_2'] == 'wikidata']['024_a'].index.to_list())

In [83]:
chybi = [x for x in opakovane if x not in autority_s_wikidaty]

In [84]:
len(chybi)

69539

In [85]:
# Wikidata SPARQL endpoint
sparql_endpoint = "https://query.wikidata.org/sparql"

# Function to get Wikidata ID from NL CR AUT ID
def get_wikidata_id(nl_cr_aut_id):
    query = f"""
    SELECT ?item WHERE {{
      ?item wdt:P691 "{nl_cr_aut_id}".
    }}
    """
    headers = {
        "Accept": "application/sparql-results+json",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    response = requests.get(sparql_endpoint, params={"query": query}, headers=headers)
    data = response.json()

    if "results" in data and "bindings" in data["results"]:
        bindings = data["results"]["bindings"]
        if bindings:
            return bindings[0]["item"]["value"].split("/")[-1]
    return None

In [86]:
co_stahovat = [x for x in chybi if x not in set(vyhledane)]

In [ ]:
pocitadlo = 0
for nl_cr_aut_id in co_stahovat:
    pocitadlo += 1
    print(f"{pocitadlo:005}: {nl_cr_aut_id}")
    slovnik = {}
    slovnik['100_7'] = nl_cr_aut_id
    wikidata_id = get_wikidata_id(nl_cr_aut_id)
    if wikidata_id:
        slovnik['024_a'] = wikidata_id
    else:
        slovnik['024_a'] = ""
    with open(os.path.join(kam, f"{nl_cr_aut_id}_{slovnik['024_a']}.json"), "w+", encoding="utf-8") as file:
        file.write(json.dumps(slovnik))

00001: jk01073373
00002: xx0022957
00003: jk01060723
00004: jx20080605009
00005: jo2002100493
00006: jo2002100492
00007: jx20061114011
00008: jn19981001999
00009: jo2009521367
00010: jk01120720
00011: jk01082976
00012: nlk19990073797
00013: xx0137864
00014: jk01120688
00015: jn20000602926
00016: xx0034029
00017: jk01110672
00018: jk01110671
00019: jk01082932
00020: jk01073318
00021: jn19981002161
00022: jk01110686
00023: mzk2002160685
00024: jn20000602810
00025: jo2001100268
00026: xx0030354
00027: jn20001103342
00028: ntka172681
00029: jk01110687
00030: jk01091596
00031: xx0084705
00032: jx20040618032
00033: xx0087542
00034: jk01141309
00035: jk01043462
00036: xx0012917
00037: jk01073336
00038: jk01092733
00039: jo20000069721
00040: mzk2003186083
00041: jn20010310068
00042: jn20010309830
00043: jo2016914683
00044: xx0113620
00045: xx0009184
00046: jk01042833
00047: jn20010309758
00048: jk01130025
00049: jk01130028
00050: ola2004212170
00051: jn19981001836
00052: xx0018390
00053: ola20